Task 2: Generating a table of transaction data of a sample of owners.

In [ ]:
import os
import re
import datetime 
from zipfile import ZipFile
import pandas as pd
import numpy as np
import pandas_gbq
import janitor
import shutil
import glob
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# Building the private key.
service_path = "C:\\Users\\rsmcd\\OneDrive\\Desktop\\MSBA Fall 2022\\" # Path to json file.
service_file = 'reese-msba-9558fdd20984.json' # Name of json file.
gbq_proj_id = 'reese-msba' # Name of project.

# Creates single variable that leads to json file.
private_key =service_path + service_file  

In [ ]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [ ]:
# And finally we establish our connection.
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
# Look at list of data sets in client.
for item in client.list_datasets() : 
    print(item.full_dataset_id)

The code in the next few cells builds the randomized owner table and writes it to csv format.

In [ ]:
# A query that grabs a list of owners, randomizes the order, selects 650 of them, and returns all transactions for them.
query = """
    WITH cnd AS(
    SELECT DISTINCT card_no
    FROM `reese-msba.wedge_transactions.transArchive*`
    WHERE card_no != 3
    ORDER BY rand()
    LIMIT 650)
    SELECT *
    FROM `reese-msba.wedge_transactions.transArchive*` as rsmba
    INNER JOIN cnd
    ON cnd.card_no = rsmba.card_no
"""

random_owners = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

In [ ]:
random_owners

In [ ]:
# Deletes the csv that will be created if it already exists in the directory.
filePath = 'sample_owner_table.csv';
    
# Check if Folder exists or not.
if os.path.exists(filePath):
    os.remove(filePath)
  
    print("The folder has been deleted successfully!")
else:
    print("Cannot delete the folder as it doesn't exists")

In [ ]:
random_owners.to_csv("sample_owner_table.csv", index=False)